# Perceptions&Interactions TP3 - Traitement d'image, Partie 2

*Imports et chargement d'image*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

img = cv2.imread("lena_gray.png")

## Exercices bonus

### 4. Features from Accelerated Segment Test (FAST)

FAST est un algorithme de détection de points d'intérêt. Pour chaque pixel, un cercle de rayon 3 des pixels "voisins" est calculé. Tout d'abord, on peut définir un tel cercle tel que :

In [ ]:
def bresenham_circle():
    circle = np.zeros((7,7))
    for i in range(circle.shape[0]):
        for j in range(circle.shape[1]):
            distance = np.sqrt((3-i)*(3-i) + (3-j)*(3-j))
            if distance > 2.5 and distance < 3.5:
                circle[i, j] = 1
    return circle
plt.imshow(bresenham_circle())

Ce cercle pourra ensuite être utilisé de manière similaire aux noyaux de la fonction convolution. Le cercle est composé de 16 pixels. Pour chaque pixel du cercle, on vérifie si sa différence d'intensité avec le pixel du centre du cercle est supérieure au threshold donné en paramètre. Si au moins 12 des 16 pixels du cercle ont une différence supérieure à ce threshold, alors le pixel est considéré comme un point d'intérêt.

La fonction fast retourne un tableau contenant n paires de coordonnées (i,j), n étant le nombre de points d'intérêts trouvés.

In [ ]:
def fast(img, threshold):
    circle = bresenham_circle()
    # ...
    return

def show_keypoints(img, keypoints):
    img_show_keypoints = np.stack((img.copy(), img.copy(), img.copy()), axis=2)
    for kp in keypoints:
        (i,j) = kp
        img_show_keypoints[i-1:i+1, j-1:j+1, 1] = 255
    plt.imshow(img_show_keypoints)
    plt.show()

In [ ]:
crew_d = cv2.imread("Crew_Dragon.jpg", cv2.IMREAD_GRAYSCALE)
capsule = cv2.imread("capsule.jpg", cv2.IMREAD_GRAYSCALE)

# Attention, prend beaucoup de temps
#kp_cd = fast(crew_d, 50)

show_keypoints(crew_d, kp_cd)

In [ ]:
# Attention, prend beaucoup de temps
kp_caps = fast(capsule, 50)

show_keypoints(capsule, kp_caps)

### 5. Binary Robust Independent Elementary Features (BRIEF)

BRIEF calcule des descripteurs pour chaque point d'intérêt. Tout d'abord, un flou est effectué sur l'image d'entrée pour diminuer le niveau de bruit. Il est possible d'utiliser un flou tel que codé précédemment avec les fonctions moyenne_voisins ou convolution, ou alors d'utiliser un flou gaussien à l'aide des deux lignes suivantes (si supporté par l'installation Python du CREMI...) :
> from scipy.ndimage import gaussian_filter
> 
> gaussian_filter(img, sigma=2)

On cherche alors à obtenir un descripteur, ici un vecteur de taille 128, pour chaque pixel détecté comme un point d'intérêt par FAST. On considère le voisinage de ce pixel (carré de taille 9x9). Dans ce voisinage, 128 paires de pixels (A,B) sont sélectionnées aléatoirement. Pour chacune de ces paires, si img(A) < img(B), alors le vecteur prend la valeur 1 (dans la case correspondant au numéro de la paire considérée). Dans le cas contraire, le vecteur prend la valeur 0.

In [ ]:
import random
def brief(img, keypoints):
    # ...
    return

desc_cd = brief(crew_d, kp_cd)
desc_caps = brief(capsule, kp_caps)

Une fois les descripteurs obtenus, ils sont comparés un à un à l'aide d'une distance de Hamming. On affiche ensuite les meilleurs matchs.

In [ ]:
# Perform the matching between the BRIEF descriptors of the training image and the test image
matches = []
for i in range(len(desc_cd)):
    for j in range(len(desc_caps)):
        matches += [cv2.DMatch(i, j, np.sum(desc_cd[i] ^ desc_caps[j])/128)]

# Order the matches in ascending order of distance
matches = sorted(matches, key = lambda x : x.distance)

# Convert keypoints to cv2 KeyPoints
kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 1) for x in kp_cd]
kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 1) for x in kp_caps]

# Display the best matching points
result = cv2.drawMatches(crew_d, kp1, capsule, kp2, matches[:30], None, flags = 2)
plt.title("Matches between the original and modified capsule images")
plt.imshow(result)
plt.show()